In [36]:
import warnings
warnings.filterwarnings("ignore") 

import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
import os, librosa
from tqdm import tqdm
import torchaudio
import IPython.display as ipd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle 
import sklearn 


In [37]:
with open('audio_data.pkl', 'rb') as f:
    audio_data_df = pickle.load(f)


In [ ]:
#split the data

from torch.utils.data import Dataset, DataLoader
from class_CNN import DatasetClass
from sklearn.model_selection import train_test_split


train_df, test_df = train_test_split(audio_data_df, test_size=0.2, random_state=42)
test_df, val_df = train_test_split(test_df, test_size=0.5, random_state=42)

# call my dataclass 

train_data = DatasetClass(train_df)
val_data = DatasetClass(val_df)
test_data = DatasetClass(test_df)

# save test data to pickle file
# with open('test_data.pkl', 'wb') as f:
#     pickle.dump(test_data, f)

#call my dataloader 

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)



In [39]:
train_data[0][0].shape

torch.Size([1, 128, 127])

In [40]:
# import model

from model_CNN import UrbanCNN
import torch

model = UrbanCNN()
optimiser = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

UrbanCNN(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (relu): ReLU()
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=31744, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

In [41]:
import wandb 
import datetime

timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M_%S')
epochs = 10
batch_size = 32


wandb.init(
    project="week5",
    name=f"flick-{timestamp}",
    config={
    "epochs": epochs,
    "batch_size": batch_size,
    "learning_rate": 0.001
})

In [42]:

for epoch in range(6):
    model.train()
    total_loss = 0

    for padded_spec, label in train_loader:
        padded_spec = padded_spec.to(device)
        label = label.to(device)

        optimiser.zero_grad()

        output = model(padded_spec)

        loss = criterion(output, label)
        loss.backward()
        optimiser.step()
        
        total_loss += loss.item()
  
    avg_train_loss = total_loss / len(train_loader)

    model.eval()
    val_loss = 0

    with torch.no_grad():
        for padded_spec, label in val_loader:
            padded_spec = padded_spec.to(device)
            label = label.to(device)

            output = model(padded_spec)
            loss = criterion(output, label)
            val_loss += loss.item()
    
    avg_val_loss = val_loss / len(val_loader)
    wandb.log({"train_loss": avg_train_loss, "val_loss": avg_val_loss})
    print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

wandb.finish()
    # print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")
    

Epoch 1, Train Loss: 3.2000, Val Loss: 1.0201
Epoch 2, Train Loss: 0.6451, Val Loss: 0.7880
Epoch 3, Train Loss: 0.3465, Val Loss: 0.8134
Epoch 4, Train Loss: 0.1923, Val Loss: 0.7946
Epoch 5, Train Loss: 0.1026, Val Loss: 0.8404


wandb: ERROR The nbformat package was not found. It is required to save notebook history.


Epoch 6, Train Loss: 0.0721, Val Loss: 0.8398


train_loss,█▂▂▁▁▁
val_loss,█▁▂▁▃▃
train_loss,0.07212
val_loss,0.83976


In [43]:
torch.save(model.state_dict(), "cnn_3couches.pt")